In [1]:
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import json
import src.geoquery as g
import src.dataPrep as f
import warnings 
warnings.filterwarnings('ignore')

# Data Visualization

#### Importamos los ficheros preparados en dataPrep para obtener las ubicaciones a tener en cuenta en nuestra elección de ubicación. Esta información se va plasmando mediante capas en un mapa en Folium para obtener un resultado más visual.

- Creamos mapa con las coordenadas de San Francisco

In [2]:
sFrancisco_coordinates = (37.773972, -122.431297)

In [3]:
#heat_m = folium.Map(location=[start_lat, start_lon],tiles='cartodbpositron', zoom_start=15)

m = folium.Map(location=sFrancisco_coordinates, tiles="OpenStreetMap")
m

- Incluimos un mapa de calor con todas las compañías tecnológicas de San Francisco. Para ello, traemos el fichero preparado anteriormente en dataPrep.

In [4]:
tech_companies = pd.read_json('OUTPUT/tech_companies')

In [5]:
sanF_group = folium.FeatureGroup(name='Sf_companies')
HeatMap(data=tech_companies[['latitude', 'longitude']]).add_to(sanF_group)
sanF_group.add_to(m)

In [6]:
m

- Obtenemos las coordenadas del aeropuerto de San Francisco mediante la API de Google Geocode

In [7]:
airport = f.googleApi('San Francisco Airport')

- Marcamos el punto en el mapa

In [8]:
uni_lon = airport['results'][0]['geometry']['location']['lng']
uni_lat = airport['results'][0]['geometry']['location']['lat']

In [9]:
Marker([uni_lat, uni_lon], tooltip=folium.Tooltip('San Francisco Airport'),
       popup=f'Coordinates: \n Lat:{uni_lat}\n Lon:{uni_lon}',
       icon=folium.Icon(color='blue', icon='plane', prefix='fa')).add_to(m)

In [10]:
m

- Incluimos marcadores verdes en una siguiente capa del mapa para identificar los starbucks. Para ello, traemos el fichero preparado anteriormente en dataPrep.

In [11]:
starbucks = pd.read_json('OUTPUT/starbucks.json')

In [12]:
for i in range(len(starbucks.latitude)):
    Circle(
        location=[starbucks.iloc[i]['latitude'],starbucks.iloc[i]['longitude']],
        radius=15,
        color='green',
        fill=True,
        fill_color='green').add_child(folium.Popup('Starbucks')).add_to(m)

In [13]:
m

- Incluimos marcadores rojos en una siguiente capa del mapa para identificar los distintos pubs, etiquetados por su nombre. Para ello, traemos el fichero preparado anteriormente en dataPrep.

In [14]:
pubs = pd.read_json('OUTPUT/pubs.json')

In [15]:
for i in range(len(pubs.latitude)):
    Circle(
        location=[pubs.iloc[i]['latitude'],pubs.iloc[i]['longitude']],
        radius=15,
        color='red',
        fill=True,
        fill_color='red').add_child(folium.Popup(pubs.iloc[i]['name'])).add_to(m)

In [16]:
m

- Incluimos marcadores amarillos en una siguiente capa del mapa para identificar los distintos colegios, etiquetados por su nombre. Para ello, traemos el fichero preparado anteriormente en dataPrep.

In [17]:
schools = pd.read_json('OUTPUT/schools.json')

In [18]:
for i in range(len(schools.latitude)):
    Circle(
        location=[schools.iloc[i]['latitude'],schools.iloc[i]['longitude']],
        radius=15,
        color='yellow',
        fill=True,
        fill_color='yellow').add_child(folium.Popup(schools.iloc[i]['name'])).add_to(m)

In [19]:
m

# Final Decision

#### Una vez situadas todas las coordenadas en el mapa, realizaremos geoqueries a MongoDB para obtener la compañía tecnológica que mejor posicionada esté respecto a los lugares de interés. Se valorará en función de la importancia y distancia, teniendo en cuenta una lejanía no superior a 1,5km. La compañía que mejor puntuación reciba, se utilizarán sus coordenadas de posición como recomendación del lugar más óptimo para la apertura de oficinas.

In [20]:
tech_comp = tech_companies[['name','category_code','number_of_employees','founded_year','latitude','longitude','location']]

- Creamos una lista con las localizaciones de las compañías para poder compararlas con los demás parámetros

In [21]:
list_techcomp = []

for i in range(len(tech_comp)):
    list_techcomp.append(tech_comp['location'][i])

- Generamos las geoqueries para cada compañía tecnológica de San Francisco para cada uno de los parámetros y así obtenemos el número de compañías que cumplen con la condición de 1,5km de cercanía. Incluimos estos resultados en el DataFrame de las compañías.

In [22]:
tech_comp['starbucks1km'] = g.geoquery(list_techcomp,'starbucks')
tech_comp['pubs1km'] = g.geoquery(list_techcomp,'pubs')
tech_comp['schools1km'] = g.geoquery(list_techcomp,'schools')

- Creamos una columna con el sumatorio de las coincidencias y puntuando según su grado de importancia, así como priorizando las ubicaciones donde existen coincidencias de los 3 parámetros (sumamos 100 puntos).

In [23]:
tech_comp['score'] = tech_comp.starbucks1km*0.5 + tech_comp.pubs1km*0.35 + tech_comp.schools1km*0.15

for row in range(len(tech_comp)):
    if tech_comp.starbucks1km[row] >= 1 and tech_comp.pubs1km[row] >= 1 and tech_comp.schools1km[row] >= 1:
        tech_comp.score[row] = tech_comp.score[row]*2

- Ordenamos de mayor a menor

In [24]:
tech_comp = tech_comp.sort_values(by=['score'], ascending = False).reset_index(drop=True)

In [25]:
tech_comp.head()

,name,category_code,number_of_employees,founded_year,latitude,longitude,location,starbucks1km,pubs1km,schools1km,score
0,PX Interactive,software,8.0,2007.0,37.781754,-122.407709,"{'type': 'Point', 'coordinates': [-122.407709,...",36,8,1,41.9
1,PX Interactive,software,8.0,2007.0,37.781754,-122.407709,"{'type': 'Point', 'coordinates': [-122.407709,...",36,8,1,41.9
2,Tiny Pictures,mobile,18.0,2005.0,37.781002,-122.406912,"{'type': 'Point', 'coordinates': [-122.406912,...",35,7,1,40.2
3,Streetline,hardware,NaN,2005.0,37.782085,-122.409747,"{'type': 'Point', 'coordinates': [-122.409747,...",31,7,1,36.2
4,Instructables,web,NaN,2005.0,37.779176,-122.405889,"{'type': 'Point', 'coordinates': [-122.4058887...",29,7,1,34.2


### Obtenemos la empresa 'PX Interactive' como la mejor ubicada, ya que satisface todas las preferencias y es la que mayor puntuación recibe al tener numerosos lugares de interés a 1,5km a la redonda. Por lo que, la decisión final es situar nuestras oficinas en dicho radio.

In [26]:
final_decisionLat = tech_comp.latitude[0]
final_decisionLon = tech_comp.longitude[0]

In [33]:
print(final_decisionLon)

-122.407709


In [27]:
Marker([final_decisionLat, final_decisionLon], tooltip=folium.Tooltip(tech_comp.name[0]),
       popup=f'Coordinates: \n Lat:{final_decisionLat}\n Lon:{final_decisionLon}',
       icon=folium.Icon(color='black', icon='briefcase', prefix='fa')).add_to(m)

In [28]:
m

In [29]:
Circle(location=[final_decisionLat, final_decisionLon],
       radius=1500,
       popup='Best offices location',
       color='black',
       fill=True,
       fill_color='black').add_to(m)

In [30]:
m

In [31]:
m.save('OUTPUT/best_location_map.html')